In [36]:
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import cv2
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import os
import random
import shutil
import numpy as np

In [6]:
# train 폴더에서 임의의 파일 20개를 뽑아 테스트로 넣어줌

folders = os.listdir("eyes_thresholded&auged/train")
for folder in folders[1:] :
    fileist = os.listdir("eyes_thresholded&auged/train"+ "/" + folder)
    testfiles = random.sample(fileist, 20)
    for i in testfiles:
        shutil.copyfile("eyes_thresholded&auged/train/"+ folder + "/" + i, "eyes_thresholded&auged/test/" + folder + "/" + i)

In [37]:
# 450개 이미지 4개로 배치 generator 생성 /  train 증식 없음
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'eyes_thresholded&auged/train',
        target_size=(100, 100),    
        batch_size=5,
        class_mode='categorical')


# target size 지정?

Found 450 images belonging to 6 classes.


In [38]:
# 120개 이미지 4개로 배치 generator 생성 /  train 증식 없음
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'eyes_thresholded&auged/test',
        target_size=(100, 100),    
        batch_size=5,
        class_mode='categorical')

Found 120 images belonging to 6 classes.


In [39]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100, 100,3)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(6, activation='softmax'))

In [40]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
model.fit_generator(
        train_generator,
        steps_per_epoch= 90,
        epochs=10,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/10
90/90 [==============================] - 15s 169ms/step - loss: 1.9793 - accuracy: 0.3333 - val_loss: 2.9368 - val_accuracy: 0.2800
Epoch 2/10
90/90 [==============================] - 16s 173ms/step - loss: 0.9035 - accuracy: 0.7022 - val_loss: 13.9575 - val_accuracy: 0.1200
Epoch 3/10
90/90 [==============================] - 15s 171ms/step - loss: 0.5111 - accuracy: 0.8400 - val_loss: 46.6164 - val_accuracy: 0.2800
Epoch 4/10
90/90 [==============================] - 16s 175ms/step - loss: 0.3582 - accuracy: 0.8956 - val_loss: 7.5789 - val_accuracy: 0.3600
Epoch 5/10
90/90 [==============================] - 15s 170ms/step - loss: 0.2158 - accuracy: 0.9533 - val_loss: 5.8026 - val_accuracy: 0.6000
Epoch 6/10
90/90 [==============================] - 15s 166ms/step - loss: 0.2288 - accuracy: 0.9422 - val_loss: 0.4159 - val_accuracy: 0.8400
Epoch 7/10
90/90 [==============================] - 15s 164ms/step - loss: 0.2147 - accuracy: 0.9333 - val_loss: 0.1512 - val_accuracy: 1.00

In [12]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
Instructions for updating:
Please use Model.evaluate, which supports generators.
accuracy: 100.00%


In [43]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'bonghwang': 0, 'dohwa': 1, 'gongjak': 2, 'horang': 3, 'so': 4, 'yong': 5}
[[0.945 0.002 0.018 0.012 0.014 0.009]
 [0.000 0.000 0.004 0.000 0.982 0.013]
 [0.000 0.000 0.000 1.000 0.000 0.000]
 [0.988 0.003 0.004 0.003 0.000 0.001]
 [0.000 0.967 0.028 0.001 0.000 0.004]
 [0.000 0.000 0.998 0.000 0.000 0.002]
 [0.000 0.001 0.974 0.000 0.000 0.025]
 [0.000 0.000 0.998 0.000 0.001 0.000]
 [0.001 0.001 0.007 0.001 0.889 0.101]
 [0.000 0.997 0.000 0.000 0.000 0.003]
 [0.000 0.001 0.065 0.001 0.930 0.003]
 [0.000 0.001 0.001 0.006 0.000 0.992]
 [0.000 0.019 0.021 0.001 0.949 0.010]
 [0.000 0.000 0.044 0.002 0.019 0.935]
 [0.000 0.890 0.107 0.000 0.001 0.002]
 [0.923 0.002 0.048 0.009 0.003 0.015]
 [0.000 0.000 0.001 0.000 0.000 0.999]
 [0.000 0.000 0.000 0.967 0.000 0.033]
 [0.002 0.069 0.146 0.001 0.592 0.190]
 [0.000 0.000 0.003 0.000 0.000 0.997]
 [0.000 0.972 0.003 0.001 0.022 0.002]
 [0.000 0.006 0.003 0.974 0.000 0.017]
 [0.000 0.000 0.000 1.000 0.000 0.000]
 [0.998 0.000

In [44]:
test_generator.class_indices

{'bonghwang': 0, 'dohwa': 1, 'gongjak': 2, 'horang': 3, 'so': 4, 'yong': 5}

# 새 이미지 추가

In [45]:
np.random.seed(5)
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [29]:
# 왼쪽 눈
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')


img = load_img('eyes_thresholded&auged/test2/1.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir='eyes_thresholded&auged/images', save_prefix='tri', save_format='jpg'):
    i += 1
    if i > 30: 
        break

In [20]:
#오른쪽 눈
img = load_img('eyes_thresholded&auged/test2/2.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in data_aug_gen.flow(x, batch_size=1, save_to_dir='eyes_thresholded&auged/images', save_prefix='tri', save_format='jpg'):
    i += 1
    if i > 30: 
        break

In [46]:
test_datagen2 = ImageDataGenerator(rescale=1./255)
test_datagen2

In [52]:
test_generator2 = test_datagen2.flow_from_directory(
        'eyes_thresholded&auged/images',
        target_size=(100, 100),    
        batch_size=2,
        class_mode='categorical')

Found 31 images belonging to 1 classes.


In [51]:
print("-- Predict --")
output = model.predict_generator(test_generator2, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(train_generator.class_indices)
print(output)

-- Predict --
{'bonghwang': 0, 'dohwa': 1, 'gongjak': 2, 'horang': 3, 'so': 4, 'yong': 5}
[[0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]
 [0.000 0.000 1.000 0.000 0.000 0.000]]


In [49]:
for i in output:
    print(i.argmax())

2
2
